<a href="https://colab.research.google.com/github/ErikVegaC/OTO-O2024/blob/main/Laboratorio_M%C3%A9todos_de_b%C3%BAsqueda.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Laboratorio: Métodos de búsqueda

En las clases anteriores creaste códigos para realizar búsquedas aleatorias (Simulated Annealing) y búsquedas dirigidas (Optimización Bayesiana). Estos métodos de búsqueda se utilizan para facilitar el proceso de optimización de funciones objetivos compleja y costosas de computar.

En este laboratorio usaremos el dataset de los diferentes tipos de iris, y sus longitudes y anchos de pétalos y sépalos. Utilizaremos un RandomForest para crear un modelo de clasificación y el métrico F1 para decidir cuál es el mejor modelo de acuerdo a lo que tenemos disponible.

1. Carga el dataset de Iris

In [2]:
from sklearn import datasets
X, y = datasets.load_iris(return_X_y=True)

2. Importa el archivo `Bosque.py`.

Este archivo contiene la función `RegresionBosque`, que recibe:
- X: las características independientes
- y: la variable de respuesta
- árboles: cantidad total de árboles
- profundidad de bosque: niveles de profundidad del bosque

Su salida es:
- modelo: El objeto con el modelo ajustado
- f1: El métrico que califica qué tan bueno es el modelo que se ajustó.


In [3]:
import Bosque
modelo, f1 = Bosque.RegresionBosque(X, y, 10, 3)
f1

0.9666666666666667

### Actividad 1:

Inicializa un espacio con 5 muestras en nuestro dominio de variables independientes:
- árboles: números enteros entre 5 y 50.
- profundidad: números enteros entre 2 y 10

Utiliza optimización Bayesiana para encontrar la combinación de árboles y profundidad que **maximice** el métrico F1.

In [25]:
import numpy as np
from sklearn.gaussian_process import GaussianProcessRegressor
from sklearn.gaussian_process.kernels import RBF
from sklearn import datasets
import Bosque


X, y = datasets.load_iris(return_X_y=True)

n = 5

arboles = np.random.randint(5, 51, n)
profundidad = np.random.randint(2, 11, n)


espacio = np.column_stack((arboles, profundidad))

lista_f1 = []

def objetivo(params):
    arboles, profundidad = params
    modelo, f1 = Bosque.RegresionBosque(X, y, arboles, profundidad)
    lista_f1.append(f1)
    return f1

for params in espacio:
    objetivo(params)

kernel = 1 * RBF(length_scale=1)


gp = GaussianProcessRegressor(kernel=kernel, n_restarts_optimizer=10).fit(espacio, lista_f1)



x_arboles = np.linspace(5, 50, 1000).reshape([-1, 1])
x_profundidad = np.linspace(2, 10, 1000).reshape([-1, 1])


X_pred = np.hstack((x_arboles, x_profundidad))

y_pred, y_std = gp.predict(X_pred, return_std=True)
y_pred_low = y_pred - 1.96 * y_std
y_pred_high = y_pred + 1.96 * y_std

i_next = np.argmax(y_pred_high)
X_next = X_pred[i_next]

def F1_opt(X_next):
  for i in range(n):
    b, p = X_next
    modelo,f1 = Bosque.RegresionBosque(X, y, b, p)
  return f1




In [26]:
X_next

array([50., 10.])

In [34]:
x11 = int(X_next[0]), int(X_next[1])

In [28]:
Bosque.RegresionBosque(X, y, 5, 2)

(RandomForestClassifier(max_depth=2, n_estimators=5), 0.9333333333333333)

In [35]:
F1_opt(x11)

0.9555555555555556

### Actividad 2:

Inicializa 2 vectores con posibles valores para las variables independientes:
- árboles: números enteros entre 5 y 50
- profundidad: números enteros entre 2 y 10

Utiliza el algoritmo de Simulated Annealing que siga el siguiente orden:
- Elige un punto de partida para las variables.
- Selecciona al azar una de las dos para modificarlas.
- Selecciona un elemento al azar de la lista que contiene los posibles valores de esa variable.
- Sigue el algoritmo ($p$ y $q$) para decidir si usas esa combinación nueva o si mantienes la anterior.

In [107]:
import numpy as np
from sklearn.gaussian_process import GaussianProcessRegressor
from sklearn.gaussian_process.kernels import RBF
from sklearn import datasets
import Bosque


X, y = datasets.load_iris(return_X_y=True)


arboles = np.random.randint(5, 51)
profundidad = np.random.randint(2, 11)

lista_f1 = []

def calcular_f1(params):
    n_arboles, profundidad = params
    modelo, f1 = Bosque.RegresionBosque(X, y, n_arboles, profundidad)
    return f1

def simulated_annealing(t_inicial, t_minima, n):
    n_arboles_actual = np.random.choice(arboles)
    profundidad_actual = np.random.choice(profundidad)
    mejor_solucion = [n_arboles_actual, profundidad_actual]
    mejor_f1 = calcular_f1(mejor_solucion)

    T = t_inicial

    while T > t_minima:
        for i in range(10):
            if np.random.random() < 0.5:
                nueva_solucion = [np.random.choice(arboles), mejor_solucion[1]]
            else:
                nueva_solucion = [mejor_solucion[0], np.random.choice(profundidad)]

            nuevo_f1 = calcular_f1(nueva_solucion)

            delta_f1 = nuevo_f1 - mejor_f1
            if delta_f1 > 0 or np.exp(delta_f1 / T) > np.random.random():
                mejor_solucion = nueva_solucion
                mejor_f1 = nuevo_f1

        T *= 0.1

    return mejor_solucion, mejor_f1




In [104]:
mejores_parametros, mejor_f1 = simulated_annealing(10, 10, 6)

In [105]:
mejores_parametros

[11, 2]

In [106]:
mejor_f1

0.9333333333333333